Importing libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import urllib.request as urllib2
from time import strptime
import re
import csv
import calendar

In [2]:
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

Initializing variables

In [3]:
url_example = "https://www.vlr.gg/search/threads/?q=f0rsaken" # URL base
# BeautifulSoup(requests.get(url_example).content, 'html.parser')

In [4]:
aPage_raw = BeautifulSoup(requests.get(url_example).content, 'html.parser').find_all("a", class_="btn mod-page") # Find all <div> tags that contains posts' texts
numPage = []
for tag in aPage_raw:
    numPage.append(int(tag.text))
print(numPage[-1])

34


In [5]:
divcontent_raw = BeautifulSoup(requests.get(url_example).content, 'html.parser').find_all("div", class_="block main") # Find all <div> tags that contains posts' texts
# divcontent_raw

In [6]:
spanVote_raw = BeautifulSoup(requests.get(url_example).content, 'html.parser').find_all("span", class_="frag-count") # Find all <div> tags that contains posts' texts
print(len(spanVote_raw))
# for tag in spanVote_raw:
#     print(" ".join(tag.text.split()))

28


In [7]:
spanComm_raw = BeautifulSoup(requests.get(url_example).content, 'html.parser').find_all("span", class_="post-count") # Find all <div> tags that contains posts' texts
# print(len(spanComm_raw))
# for tag in spanComm_raw:
#     print(tag.text.split()[0])

In [8]:
soupCommTread = BeautifulSoup(requests.get('https://www.vlr.gg/154605/skrossi-lf-vs-f0rsaken-right-now').content, 'html.parser')
soupEdit = soupCommTread.find_all('span', class_="post-edit")

In [9]:
for edit in soupEdit:
    edit.span.decompose()

In [10]:
iFlagDate_raw = soupCommTread.find_all('i', class_="post-header-flag")
iFlagComm_TempLst = []
for tag in iFlagDate_raw:
    iFlagComm_TempLst.append(tag.get("title"))

print(len(iFlagComm_TempLst))


123


In [11]:
soupTemp = BeautifulSoup(requests.get('https://www.vlr.gg/34947/vision-strikers-vs-paper-rex-valorant-champions-tour-stage-3-masters-berlin-opening-a').content, 'html.parser')
soupTemp.find_all('a', class_="match-header-event")

[<a class="match-header-event" href="/event/466/valorant-champions-tour-stage-3-masters-berlin/group-stage">
 <img src="//owcdn.net/img/603bfd7bf3f54.png" style="height: 32px; width: 32px; margin-right: 6px;"/>
 <div>
 <div style="font-weight: 700;">
 						Valorant Champions Tour Stage 3: Masters Berlin					</div>
 <div class="match-header-event-series">
 						Group Stage: 
 						Opening (A)					</div>
 </div>
 </a>]

In [12]:
soupTemp = BeautifulSoup(requests.get('https://www.vlr.gg/389781/f0rsaken-elaborates-on-difficulty-facing-edg-in-press-conference').content, 'html.parser')
soupTemp.find_all('a', class_='article-meta-author')

[<a class="article-meta-author" href="/user/Seulgi" style="margin-top: 2px;">Seulgi</a>]

In [46]:
def beautifulSoupWebScrapeVLRComments(urlBase, 
                                      plyrName, 
                                      loopCount, 
                                      threadTitleLst_plyr, 
                                      threadTypeLst_plyr,  
                                      commLst_plyr,  
                                      commFlagLst_plyr,
                                      commVoteLst_plyr,
                                      commDateLst_plyr,
                                      commTimeLst_plyr,  
                                      threadDateLst_plyr, 
                                      threadPostLst_plyr):
    print("Collecting comments for: " + plyrName + "\n")
    for i in range(loopCount):
        url_extra = "&page="
        url_final = urlBase + plyrName
        if i != 0:
            url_final += url_extra + str(i+1)
        
        print(url_final)
        
        soupRes = BeautifulSoup(requests.get(url_final).content, 'html.parser')
    
        aType_raw = soupRes.find_all("a") # Find all <div> tags that contains posts' texts
        typeDisc = []
        for tag in aType_raw:
            printText = " ".join(tag.text.split())
            if printText == "General Discussion" or printText == "Off Topic" or printText == "Matches" or printText == "News":
                typeDisc.append(printText)
        typeDisc.pop(0)

        aTitle_raw = soupRes.find_all("a", class_="thread-item-header-title") # Find all <div> tags that contains posts' texts
        spanDate_raw = soupRes.find_all("span", class_="date-full hide")
        
        titleLst = []
        for tag in aTitle_raw:
            titleLst.append(" ".join(tag.text.split()))
        
        dateLst = []
        for tag in spanDate_raw:
            datetimeVLRPost = [a.strip().replace(',', '') for a in " ".join(tag.text.split()).split("at")]
            month = list(calendar.month_name).index(datetimeVLRPost[0].split(" ")[0])
            date = datetimeVLRPost[0].split(" ")[1]
            year = datetimeVLRPost[0].split(" ")[-1]
            datePost = date+"/"+str(month)+"/"+year
            dateLst.append(datePost)

        timeLst = []
        for tag in spanDate_raw:
            datetimeVLRPost = [a.strip().replace(',', '') for a in " ".join(tag.text.split()).split("at")]
            hour = int(datetimeVLRPost[1].split(" ")[0].split(":")[0])
            minute = datetimeVLRPost[1].split(" ")[0].split(":")[1]
            if datetimeVLRPost[1].split(" ")[1] == "PM" and hour != 12:
                hour += 12
            timePost = str(hour)+":"+minute
            timeLst.append(timePost)

        typeDiscWithVotes = []
        titleLstWithVotes = []
        dateLstWithVotes = []
        timeLstWithVotes = []

        for j in range(len(typeDisc)):
            if typeDisc[j] == "Matches" or typeDisc[j] == "News":
                continue
            else:
                typeDiscWithVotes.append(typeDisc[j])
                titleLstWithVotes.append(titleLst[j])
                dateLstWithVotes.append(dateLst[j])
                timeLstWithVotes.append(timeLst[j])

        commThreads_lst = []
        commVoteThread_lst = []
        dateCommThreadLst = []
        timeCommThreadLst = []
        flagCommThreadLst = []

        print("\nAnalyzing thread comments...")
        for tags in aTitle_raw:
            threadLink = "https://www.vlr.gg" + tags.get('href')
            soupThread = BeautifulSoup(requests.get(threadLink).content, 'html.parser')
            soupDetectMatch = soupThread.find_all('a', class_="match-header-event")
            soupDetectNews = soupThread.find_all('a', class_='article-meta-author')
            if len(soupDetectMatch) == 1 or len(soupDetectNews) == 1:
                print(threadLink + " will not be added")
                continue

            commTemp_lst = []
            voteTemp_lst = []
            dateTemp_lst = []
            timeTemp_lst = []
            flagTemp_lst = []
            
            soupEdit = soupThread.find_all('span', class_="post-edit")
            for edit in soupEdit:
                edit.span.decompose()
                
            spanCommDate_raw = soupThread.find_all('span', class_="js-date-toggle")
            for tag in spanCommDate_raw:
                dateComm = tag.get('title').split("at")[0].strip().replace(",", "")
                month = list(calendar.month_abbr).index(dateComm.split(" ")[0])
                date = dateComm.split(" ")[1]
                year = dateComm.split(" ")[-1]
                datePost = date+"/"+str(month)+"/"+year

                # timeComm = k.split("at")[1].strip()
                timeComm = tag.get('title').split("at")[1].strip()
                hour = int(timeComm.split(" ")[0].split(":")[0])
                minute = timeComm.split(" ")[0].split(":")[1]
                if timeComm.split(" ")[1] == "PM" and hour != 12:
                    hour += 12
                timePost = str(hour)+":"+minute
                
                dateTemp_lst.append(datePost)
                timeTemp_lst.append(timePost)
            
            commThreads_raw = soupThread.find_all('div', class_='post-body')
#             commThreads_raw
            for tag in commThreads_raw:
                commTemp_lst.append(deEmojify(re.sub(r'http\S+', '', " ".join(tag.text.split())).strip()))
#             print(len(commThreads_lst))

            flagCommThread_raw = soupThread.find_all('i', class_="post-header-flag")
            for tag in flagCommThread_raw:
                flagTemp_lst.append(tag.get("title"))
            
            commVoteThread_raw = soupThread.find_all('div', class_='post-frag-count')
#             commVoteThread_raw
            for tag in commVoteThread_raw:
                voteTemp_lst.append(" ".join(tag.text.split()))
#             print(len(commVoteThread_lst))
            
            commThreadHeadVote_raw = soupThread.find_all('div', id="thread-frag-count")
            commThreadHeadVote = list()
            for tag in commThreadHeadVote_raw:
                commThreadHeadVote.append(int(" ".join(tag.text.split())))
#           print(commThreadHeadVote[0])

            voteTemp_lst.insert(0, commThreadHeadVote[0])
            # print(len(voteTemp_lst))
            
            commThreads_lst.append(commTemp_lst)
            flagCommThreadLst.append(flagTemp_lst)
            commVoteThread_lst.append(voteTemp_lst)
            dateCommThreadLst.append(dateTemp_lst)
            timeCommThreadLst.append(timeTemp_lst)
        
        print("Submitting all comments...")

        for j in range(len(typeDiscWithVotes)):
            for k in range(len(commThreads_lst[j])):

                # print(titleLstWithVotes[j])
                threadTitleLst_plyr.append(titleLstWithVotes[j])

                # print(typeDiscWithVotes[j])
                threadTypeLst_plyr.append(typeDiscWithVotes[j])

                # print(commThreads_lst[j][k])
                commLst_plyr.append(commThreads_lst[j][k])

                # print(flagCommThreadLst[j][k])
                commFlagLst_plyr.append(flagCommThreadLst[j][k])

                # print(commVoteThread_lst[j][k])
                commVoteLst_plyr.append(commVoteThread_lst[j][k])

                # print(dateCommThreadLst[j][k])
                commDateLst_plyr.append(dateCommThreadLst[j][k])

                # print(timeCommThreadLst[j][k])
                commTimeLst_plyr.append(timeCommThreadLst[j][k])

                # print(dateLstWithVotes[j])
                threadDateLst_plyr.append(dateLstWithVotes[j])

                # print(timeLstWithVotes[j])
                threadPostLst_plyr.append(timeLstWithVotes[j])

        print("\n")
    print("Collecting comments for " + plyrName + " is completed\n")

In [47]:
# URL
url_base = "https://www.vlr.gg/search/threads/?q=" # URL base
playerNames = ['f0rsaken', 'jinggg', 'd4v41', 'mindfreak', 'something', 'benkai']
# playerNames = ['f0rsaken']

# for loop parts
loopCount_list = []
for i in range(len(playerNames)):
    url_example = url_base + playerNames[i]
    aPage_raw = BeautifulSoup(requests.get(url_example).content, 'html.parser').find_all("a", class_="btn mod-page") # Find all <div> tags that contains posts' texts
    numPage = []
    for tag in aPage_raw:
        numPage.append(int(tag.text))
    loopCount_list.append(numPage[-1])

print(loopCount_list)

# Lists
titleThread_finalLst = []
for i in range(len(playerNames)):
    titleThread_finalLst.append(list())
    
threadType_finalLst = []
for i in range(len(playerNames)):
    threadType_finalLst.append(list())
    
comm_finalLst = []
for i in range(len(playerNames)):
    comm_finalLst.append(list())
    
flagComm_finalLst = []
for i in range(len(playerNames)):
    flagComm_finalLst.append(list())
    
voteComm_finalLst = []
for i in range(len(playerNames)):
    voteComm_finalLst.append(list())
    
dateComm_finalLst = []
for i in range(len(playerNames)):
    dateComm_finalLst.append(list())
    
timeComm_finalLst = []
for i in range(len(playerNames)):
    timeComm_finalLst.append(list())
    
dateThread_finalLst = []
for i in range(len(playerNames)):
    dateThread_finalLst.append(list())
    
timeThread_finalLst = []
for i in range(len(playerNames)):
    timeThread_finalLst.append(list())
    
print(titleThread_finalLst)
print(threadType_finalLst)
print(comm_finalLst)
print(flagComm_finalLst)
print(voteComm_finalLst)
print(dateComm_finalLst)
print(timeComm_finalLst)
print(dateThread_finalLst)
print(timeThread_finalLst)

[34, 34, 34, 34, 34, 34]
[[], [], [], [], [], []]
[[], [], [], [], [], []]
[[], [], [], [], [], []]
[[], [], [], [], [], []]
[[], [], [], [], [], []]
[[], [], [], [], [], []]
[[], [], [], [], [], []]
[[], [], [], [], [], []]
[[], [], [], [], [], []]


In [48]:
for i in range(len(playerNames)):
    beautifulSoupWebScrapeVLRComments(url_base, 
                              playerNames[i], 
                              loopCount_list[i], 
                              titleThread_finalLst[i], 
                              threadType_finalLst[i], 
                              comm_finalLst[i],  
                              flagComm_finalLst[i],
                              voteComm_finalLst[i], 
                              dateComm_finalLst[i], 
                              timeComm_finalLst[i], 
                              dateThread_finalLst[i], 
                              timeThread_finalLst[i])


https://www.vlr.gg/search/threads/?q=f0rsaken

Analyzing thread comments...
https://www.vlr.gg/408090/team-f0rsaken-vs-team-xccurate-champions-tour-2024-pacific-ascension-main-event will not be added
Submitting all comments...


https://www.vlr.gg/search/threads/?q=f0rsaken&page=2

Analyzing thread comments...
https://www.vlr.gg/389781/f0rsaken-elaborates-on-difficulty-facing-edg-in-press-conference will not be added
https://www.vlr.gg/34947/vision-strikers-vs-paper-rex-valorant-champions-tour-stage-3-masters-berlin-opening-a will not be added
Submitting all comments...


https://www.vlr.gg/search/threads/?q=f0rsaken&page=3

Analyzing thread comments...
https://www.vlr.gg/106920/paper-rex-vs-team-secret-champions-tour-asia-pacific-stage-2-challengers-playoffs-ubf will not be added
Submitting all comments...


https://www.vlr.gg/search/threads/?q=f0rsaken&page=4

Analyzing thread comments...
https://www.vlr.gg/39492/paper-rex-vs-global-esports-champions-tour-asia-pacific-last-chance-qu

In [50]:
filename_head = "data_comm_"
filename_tail = "_rep_view_final.csv"
for i in range(len(playerNames)):
    filename_final = filename_head+playerNames[i].lower()+filename_tail
    with open(filename_final, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for j in range(len(titleThread_finalLst[i])):
            writer.writerow([titleThread_finalLst[i][j], 
                             threadType_finalLst[i][j],  
                             comm_finalLst[i][j],
                             flagComm_finalLst[i][j],
                             voteComm_finalLst[i][j], 
                             dateComm_finalLst[i][j],
                             timeComm_finalLst[i][j], 
                             dateThread_finalLst[i][j],
                             timeThread_finalLst[i][j]
                            ])

In [17]:
# for tag in aTitle_raw:
#     print(" ".join(tag.text.split()))

In [18]:
# spanDate_raw = BeautifulSoup(requests.get(url_example).content, 'html.parser').find_all("span", class_="date-full hide")
# spanDate_raw

In [19]:
# for tag in spanDate_raw:
#     datetimeVLRPost = [a.strip().replace(',', '') for a in " ".join(tag.text.split()).split("at")]
#     month = list(calendar.month_name).index(datetimeVLRPost[0].split(" ")[0])
#     date = datetimeVLRPost[0].split(" ")[1]
#     year = datetimeVLRPost[0].split(" ")[-1]
#     datePost = str(month)+"/"+date+"/"+year
    
#     hour = int(datetimeVLRPost[1].split(" ")[0].split(":")[0])
#     minute = datetimeVLRPost[1].split(" ")[0].split(":")[1]
#     if datetimeVLRPost[1].split(" ")[1] == "PM":
#         hour += 12
#     timePost = str(hour)+":"+minute
#     print(timePost)
#     print(datePost, timePost)
#     print(datetimeVLRPost)

In [20]:
def beautifulSoupWebScrapeVLR(urlBase, 
                              plyrName, 
                              loopCount, 
                              titleLst_plyr, 
                              typeLst_plyr, 
                              voteLst_plyr, 
                              commLst_plyr, 
                              datePostLst_plyr, 
                              timePostLst_plyr):
    for i in range(loopCount):
        url_extra = "&page="
        url_final = urlBase + plyrName
        if i != 0:
            url_final += url_extra + str(i+1)
            
        print("Analyzing " + plyrName + "-related threads...")
        
        soupRes = BeautifulSoup(requests.get(url_final).content, 'html.parser')
    
        aTest_raw = soupRes.find_all("a") # Find all <div> tags that contains posts' texts
        typeDisc = []
        for tag in aTest_raw:
            printText = " ".join(tag.text.split())
            if printText == "General Discussion" or printText == "Off Topic" or printText == "Matches" or printText == "News":
                typeDisc.append(printText)
        typeDisc.pop(0)

        aTitle_raw = soupRes.find_all("a", class_="thread-item-header-title") # Find all <div> tags that contains posts' texts    
        spanVote_raw = soupRes.find_all("span", class_="frag-count") # Find all <div> tags that contains posts' texts
        spanComm_raw = soupRes.find_all("span", class_="post-count") # Find all <div> tags that contains posts' texts
        spanDate_raw = soupRes.find_all("span", class_="date-full hide")

        count = 0
        for j in range(len(typeDisc)):
            if typeDisc[j] == "Matches" or typeDisc[j] == "News":
                continue
            else:
                titleLst_plyr.append(" ".join(aTitle_raw[j].text.split()))
                
                typeLst_plyr.append(typeDisc[j])
                
                commLst_plyr.append(" ".join(spanComm_raw[j].text.split()).split()[0])
                
                datetimeVLRPost = [a.strip().replace(',', '') for a in " ".join(spanDate_raw[j].text.split()).split("at")]
                month = list(calendar.month_name).index(datetimeVLRPost[0].split(" ")[0])
                date = datetimeVLRPost[0].split(" ")[1]
                year = datetimeVLRPost[0].split(" ")[-1]
                datePost = date+"/"+str(month)+"/"+year
                datePostLst_plyr.append(datePost)
                
                hour = int(datetimeVLRPost[1].split(" ")[0].split(":")[0])
                minute = datetimeVLRPost[1].split(" ")[0].split(":")[1]
                if datetimeVLRPost[1].split(" ")[1] == "PM" and hour != 12:
                    hour += 12
                timePost = str(hour)+":"+minute
                timePostLst_plyr.append(timePost)
                
                count += 1
        
        for j in range(count):
            voteLst_plyr.append(int(" ".join(spanVote_raw[j].text.split())))
            
        print("Analyzing " + plyrName + "-related threads done...\n")

In [21]:
# URL
url_base = "https://www.vlr.gg/search/threads/?q=" # URL base
playerNames = ['f0rsaken', 'jinggg', 'd4v41', 'mindfreak', 'something', 'benkai']
# playerNames = ['f0rsaken']
# url_add = "&page=" # URL additional for more than 30 posts

# for loop parts
loopCount_list = []
for i in range(len(playerNames)):
    url_example = url_base + playerNames[i]
    aPage_raw = BeautifulSoup(requests.get(url_example).content, 'html.parser').find_all("a", class_="btn mod-page") # Find all <div> tags that contains posts' texts
    numPage = []
    for tag in aPage_raw:
        numPage.append(int(tag.text))
    loopCount_list.append(numPage[-1])

print(loopCount_list)

# Lists
titleLst_final_list = []
for i in range(len(playerNames)):
    titleLst_final_list.append(list())
    
typeLst_final_list = []
for i in range(len(playerNames)):
    typeLst_final_list.append(list())
    
voteLst_final_list = []
for i in range(len(playerNames)):
    voteLst_final_list.append(list())
    
commLst_final_list = []
for i in range(len(playerNames)):
    commLst_final_list.append(list())
    
datePostLst_final_list = []
for i in range(len(playerNames)):
    datePostLst_final_list.append(list())
    
timePostLst_final_list = []
for i in range(len(playerNames)):
    timePostLst_final_list.append(list())
    
print(titleLst_final_list)
print(typeLst_final_list)
print(voteLst_final_list)
print(commLst_final_list)
print(datePostLst_final_list)
print(timePostLst_final_list)

[34, 34, 34, 34, 34, 34]
[[], [], [], [], [], []]
[[], [], [], [], [], []]
[[], [], [], [], [], []]
[[], [], [], [], [], []]
[[], [], [], [], [], []]
[[], [], [], [], [], []]


In [22]:
for i in range(len(playerNames)):
    beautifulSoupWebScrapeVLR(url_base, 
                              playerNames[i], 
                              loopCount_list[i], 
                              titleLst_final_list[i], 
                              typeLst_final_list[i], 
                              voteLst_final_list[i], 
                              commLst_final_list[i], 
                              datePostLst_final_list[i], 
                              timePostLst_final_list[i])

Analyzing f0rsaken-related threads...
Analyzing f0rsaken-related threads done...

Analyzing f0rsaken-related threads...
Analyzing f0rsaken-related threads done...

Analyzing f0rsaken-related threads...
Analyzing f0rsaken-related threads done...

Analyzing f0rsaken-related threads...
Analyzing f0rsaken-related threads done...

Analyzing f0rsaken-related threads...
Analyzing f0rsaken-related threads done...

Analyzing f0rsaken-related threads...
Analyzing f0rsaken-related threads done...

Analyzing f0rsaken-related threads...
Analyzing f0rsaken-related threads done...

Analyzing f0rsaken-related threads...
Analyzing f0rsaken-related threads done...

Analyzing f0rsaken-related threads...
Analyzing f0rsaken-related threads done...

Analyzing f0rsaken-related threads...
Analyzing f0rsaken-related threads done...

Analyzing f0rsaken-related threads...
Analyzing f0rsaken-related threads done...

Analyzing f0rsaken-related threads...
Analyzing f0rsaken-related threads done...

Analyzing f0rsak

In [23]:
# titleLst_final_noDup_list = [] # List of posts' text with no duplicates
# typeLst_final_nodup_list = [] # List of posts' replies number with no duplicates
# voteLst_final_noDup_list = [] # List of posts' views number with no duplicates
# commLst_final_noDup_list = [] # List of posts' subforum titles with no duplicates
# datePostLst_final_noDup_list = [] # List of posts' topic titles with no duplicate
# timePostLst_final_noDup_list = [] # List of posts' topic titles with no duplicate
# for i in range(len(playerNames)):
#     titleLst_final_noDup_list.append(list())
#     typeLst_final_nodup_list.append(list())
#     voteLst_final_noDup_list.append(list())
#     commLst_final_noDup_list.append(list())
#     datePostLst_final_noDup_list.append(list())
#     timePostLst_final_noDup_list.append(list())
    
# for i in range(len(playerNames)):
#     for j in range(len(titleLst_final_list[i])):
#         if titleLst_final_list[i][j] not in titleLst_final_noDup_list[i]: # if text does not exist in the no-duplicate list
#             # Added to the posts' texts' no-duplicate list
#             titleLst_final_noDup_list[i].append(titleLst_final_list[i][j])
#             # Added to the posts' replies numbers' no-duplicate list
#             typeLst_final_nodup_list[i].append(typeLst_final_list[i][j])
#             # Added to the posts' views numbers' no-duplicate list
#             voteLst_final_noDup_list[i].append(voteLst_final_list[i][j])
#             # Added to the posts' subforum titles' no-duplicate list
#             commLst_final_noDup_list[i].append(commLst_final_list[i][j])
#             # Added to the posts' topic titles' no-duplicate list
#             datePostLst_final_noDup_list[i].append(datePostLst_final_list[i][j])
#             # Added to the posts' topic titles' no-duplicate list
#             timePostLst_final_noDup_list[i].append(timePostLst_final_list[i][j])

In [24]:
# filename_head = "data_"
# filename_tail = "_rep_view_final.csv"
# for i in range(len(playerNames)):
#     filename_final = filename_head+playerNames[i].lower()+filename_tail
#     with open(filename_final, 'w', newline='', encoding='utf-8') as file:
#         writer = csv.writer(file)
#         for j in range(len(titleLst_final_noDup_list[i])):
#             writer.writerow([titleLst_final_noDup_list[i][j], 
#                              typeLst_final_nodup_list[i][j], 
#                              voteLst_final_noDup_list[i][j], 
#                              commLst_final_noDup_list[i][j], 
#                              datePostLst_final_noDup_list[i][j],
#                              timePostLst_final_noDup_list[i][j]
#                             ])

In [25]:
filename_head = "data_"
filename_tail = "_rep_view_final.csv"
for i in range(len(playerNames)):
    filename_final = filename_head+playerNames[i].lower()+filename_tail
    with open(filename_final, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for j in range(len(titleLst_final_list[i])):
            writer.writerow([titleLst_final_list[i][j], 
                             typeLst_final_list[i][j], 
                             voteLst_final_list[i][j], 
                             commLst_final_list[i][j], 
                             datePostLst_final_list[i][j],
                             timePostLst_final_list[i][j]
                            ])

In [42]:
def beautifulSoupWebScrapeVLRMatch(urlBase, 
                                   plyrName, 
                                   loopCount, 
                                   titleMatchLst_plyr,
                                   divisionMatchLst_plyr,
                                   typeMatchLst_plyr,
                                   resMatchLst_plyr, 
                                   teamMatchLst_plyr, 
                                   oppMatchLst_plyr,
                                   teamScoreLst_plyr,
                                   oppScoreLst_plyr, 
                                   dateMatchLst_plyr, 
                                   timeMatchLst_plyr):
    print("Analyzing " + plyrName + "'s matches...\n")
    for i in range(loopCount):
        url_extra = "/?page="
        url_final = urlBase
        if i != 0:
            url_final += url_extra + str(i+1)
        
        print(url_final)
        
        soupRes = BeautifulSoup(requests.get(url_final).content, 'html.parser')
        aMatch_raw = soupRes.find_all("a", class_="wf-card fc-flex m-item")

        iTemps = soupRes.find_all("i", class_="fa")
        for iTemp in iTemps:
            iTemp.decompose()
            
        matchRes_lst = []
        matchTeamScore_lst = []
        matchOppScore_lst = []
        matchTeamName_lst = []
        matchOppName_lst = []
        matchDate_lst = []
        
        matchTime_lst = []
        matchDiv_raw = soupRes.find_all("div", class_="m-item-result")
        for matchRes in matchDiv_raw:
            matchRes_lst.append(matchRes.get("class")[1].split("-")[1])
            matchScores = matchRes.find_all("span")
            for count in range(len(matchScores)):
                if count % 2:
                    matchOppScore_lst.append(matchScores[count].text)
                    # matchOppName_lst.append(" ".join(teamNames[count].text.split()))
                else:
                    matchTeamScore_lst.append(matchScores[count].text)
                    # matchTeamName_lst.append(" ".join(teamNames[count].text.split()))
                    
        
        teamNames = soupRes.find_all("span", class_="m-item-team-name")
        for count in range(len(teamNames)):
            if count % 2:
                matchOppName_lst.append(" ".join(teamNames[count].text.split()))
            else:
                matchTeamName_lst.append(" ".join(teamNames[count].text.split()))
            
            
                    
        matchDate_raw = soupRes.find_all("div", class_="m-item-date")
        for match in matchDate_raw:
            match.div.unwrap()
        for match in matchDate_raw:
            matchDatetime = " ".join(match.text.split())
            matchDate = matchDatetime.split(" ")[0].split("/")[2]
            matchMonth = matchDatetime.split(" ")[0].split("/")[1]
            matchYear = matchDatetime.split(" ")[0].split("/")[0]
            matchDate_lst.append(matchDate + "/" + matchMonth + "/" + matchYear)
            
            matchHour = int(matchDatetime.split(" ")[1].split(":")[0])
            matchMin = matchDatetime.split(" ")[1].split(":")[1]
            if matchDatetime.split(" ")[2] == "pm" and matchHour != 12:
                matchHour += 12
            
            matchTime_lst.append(str(matchHour) + ":" + matchMin)
        
        for count in range(len(aMatch_raw)):
            linkMatch = "https://www.vlr.gg" + aMatch_raw[count].get('href')
            soupMatch = BeautifulSoup(requests.get(linkMatch).content, 'html.parser')
            titleMatch = [txt.strip() for txt in " ".join(soupMatch.find_all('title')[0].text.split()).split("|")]
            
            titleMatchLst_plyr.append(titleMatch[0])
            divisionMatchLst_plyr.append(titleMatch[1])
            typeMatchLst_plyr.append(titleMatch[2])
            resMatchLst_plyr.append(matchRes_lst[count])
            teamMatchLst_plyr.append(matchTeamName_lst[count])
            oppMatchLst_plyr.append(matchOppName_lst[count])
            teamScoreLst_plyr.append(matchTeamScore_lst[count])
            oppScoreLst_plyr.append(matchOppScore_lst[count])
            dateMatchLst_plyr.append(matchDate_lst[count])
            timeMatchLst_plyr.append(matchTime_lst[count])
            
    print("Analyzing " + plyrName + "'s matches done...\n")

In [43]:
# URL
playerNames = ['f0rsaken', 'jinggg', 'd4v41', 'mindfreak', 'something', 'benkai']
# playerNames = ['f0rsaken']
prxProf_lst = []

# f0rsakeN link
vlrLink_f0rsakeN = "https://www.vlr.gg/player/matches/9801/f0rsaken"
prxProf_lst.append(vlrLink_f0rsakeN)

# jinggg link
vlrLink_jinggg = "https://www.vlr.gg/player/matches/7378/jinggg"
prxProf_lst.append(vlrLink_jinggg)

# d4v41 link
vlrLink_d4v41 = "https://www.vlr.gg/player/matches/9803/d4v41"
prxProf_lst.append(vlrLink_d4v41)

# mindfreak link
vlrLink_mindfreak = "https://www.vlr.gg/player/matches/9800/mindfreak"
prxProf_lst.append(vlrLink_mindfreak)

# something link
vlrLink_something = "https://www.vlr.gg/player/matches/17086/something"
prxProf_lst.append(vlrLink_something)

# benkai link
vlrLink_benkai = "https://www.vlr.gg/player/matches/9802/benkai"
prxProf_lst.append(vlrLink_benkai)

# for loop parts
loopCount_list = []
for i in range(len(playerNames)):
    url_example = prxProf_lst[i]
    aPage_raw = BeautifulSoup(requests.get(url_example).content, 'html.parser').find_all("a", class_="btn mod-page") # Find all <div> tags that contains posts' texts
    numPage = []
    for tag in aPage_raw:
        numPage.append(int(tag.text))
    loopCount_list.append(numPage[-1])

print(loopCount_list)

# Lists
titleMatch_finalLst = []
for i in range(len(playerNames)):
    titleMatch_finalLst.append(list())
    
divisionMatch_finalLst = []
for i in range(len(playerNames)):
    divisionMatch_finalLst.append(list())
    
typeMatch_finalLst = []
for i in range(len(playerNames)):
    typeMatch_finalLst.append(list())
    
resMatch_finalLst = []
for i in range(len(playerNames)):
    resMatch_finalLst.append(list())
    
teamMatch_finalLst = []
for i in range(len(playerNames)):
    teamMatch_finalLst.append(list())
    
oppMatch_finalLst = []
for i in range(len(playerNames)):
    oppMatch_finalLst.append(list())
    
teamScoreMatch_finalLst = []
for i in range(len(playerNames)):
    teamScoreMatch_finalLst.append(list())
    
oppScoreMatch_finalLst = []
for i in range(len(playerNames)):
    oppScoreMatch_finalLst.append(list())
    
dateMatch_finalLst = []
for i in range(len(playerNames)):
    dateMatch_finalLst.append(list())
    
timeMatch_finalLst = []
for i in range(len(playerNames)):
    timeMatch_finalLst.append(list())
    
print(titleMatch_finalLst)
print(divisionMatch_finalLst)
print(typeMatch_finalLst)
print(resMatch_finalLst)
print(teamMatch_finalLst)
print(oppMatch_finalLst)
print(teamScoreMatch_finalLst)
print(oppScoreMatch_finalLst)
print(dateMatch_finalLst)
print(timeMatch_finalLst)

[5, 4, 5, 5, 3, 4]
[[], [], [], [], [], []]
[[], [], [], [], [], []]
[[], [], [], [], [], []]
[[], [], [], [], [], []]
[[], [], [], [], [], []]
[[], [], [], [], [], []]
[[], [], [], [], [], []]
[[], [], [], [], [], []]
[[], [], [], [], [], []]
[[], [], [], [], [], []]


In [44]:
for i in range(len(playerNames)):
    # print(prxProf_lst[i])
    beautifulSoupWebScrapeVLRMatch(prxProf_lst[i], 
                                   playerNames[i], 
                                   loopCount_list[i], 
                                   titleMatch_finalLst[i], 
                                   divisionMatch_finalLst[i], 
                                   typeMatch_finalLst[i], 
                                   resMatch_finalLst[i], 
                                   teamMatch_finalLst[i], 
                                   oppMatch_finalLst[i], 
                                   teamScoreMatch_finalLst[i], 
                                   oppScoreMatch_finalLst[i], 
                                   dateMatch_finalLst[i], 
                                   timeMatch_finalLst[i])

Analyzing f0rsaken's matches...

https://www.vlr.gg/player/matches/9801/f0rsaken
https://www.vlr.gg/player/matches/9801/f0rsaken/?page=2
https://www.vlr.gg/player/matches/9801/f0rsaken/?page=3
https://www.vlr.gg/player/matches/9801/f0rsaken/?page=4
https://www.vlr.gg/player/matches/9801/f0rsaken/?page=5
Analyzing f0rsaken's matches done...

Analyzing jinggg's matches...

https://www.vlr.gg/player/matches/7378/jinggg
https://www.vlr.gg/player/matches/7378/jinggg/?page=2
https://www.vlr.gg/player/matches/7378/jinggg/?page=3
https://www.vlr.gg/player/matches/7378/jinggg/?page=4
Analyzing jinggg's matches done...

Analyzing d4v41's matches...

https://www.vlr.gg/player/matches/9803/d4v41
https://www.vlr.gg/player/matches/9803/d4v41/?page=2
https://www.vlr.gg/player/matches/9803/d4v41/?page=3
https://www.vlr.gg/player/matches/9803/d4v41/?page=4
https://www.vlr.gg/player/matches/9803/d4v41/?page=5
Analyzing d4v41's matches done...

Analyzing mindfreak's matches...

https://www.vlr.gg/player/

In [45]:
filename_head = "data_match_"
filename_tail = "_rep_view_final.csv"
for i in range(len(playerNames)):
    filename_final = filename_head+playerNames[i].lower()+filename_tail
    with open(filename_final, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for j in range(len(titleMatch_finalLst[i])):
            writer.writerow([titleMatch_finalLst[i][j], 
                             divisionMatch_finalLst[i][j],
                             typeMatch_finalLst[i][j], 
                             resMatch_finalLst[i][j], 
                             teamMatch_finalLst[i][j],  
                             oppMatch_finalLst[i][j], 
                             teamScoreMatch_finalLst[i][j],  
                             oppScoreMatch_finalLst[i][j],
                             dateMatch_finalLst[i][j],
                             timeMatch_finalLst[i][j]
                            ])

In [30]:

aPage_raw = BeautifulSoup(requests.get(vlrLink_f0rsakeN).content, 'html.parser').find_all("a", class_="btn mod-page") # Find all <div> tags that contains posts' texts
aPage_raw

[<a class="btn mod-page" href="/player/matches/9801/f0rsaken/?page=2">2</a>,
 <a class="btn mod-page" href="/player/matches/9801/f0rsaken/?page=3">3</a>,
 <a class="btn mod-page" href="/player/matches/9801/f0rsaken/?page=4">4</a>,
 <a class="btn mod-page" href="/player/matches/9801/f0rsaken/?page=5">5</a>]

In [31]:
linkMatches_lst = [list() for i in prxProf_lst]
print(linkMatches_lst)
for i in range(len(prxProf_lst)):
    soupPrxPlyr = BeautifulSoup(requests.get(prxProf_lst[i]).content, 'html.parser')
    aMatch_raw = soupPrxPlyr.find_all("a", class_="wf-card fc-flex m-item")
    for tag in aMatch_raw:
        linkMatch = "https://www.vlr.gg" + tag.get('href')
        linkMatches_lst[i].append(linkMatch)

print(len(linkMatches_lst[0]), len(linkMatches_lst[1]), len(linkMatches_lst[2]), len(linkMatches_lst[3]), len(linkMatches_lst[4]), len(linkMatches_lst[5]))

[[], [], [], [], [], []]
50 50 50 50 50 50


In [32]:
soupPrxPlyr = BeautifulSoup(requests.get(prxProf_lst[0]).content, 'html.parser')
matchDate = soupPrxPlyr.find_all("div", class_="m-item-date")
for match in matchDate:
    match.div.unwrap()
    
dateMatch = []
timeMatch = []
print(len(matchDate))
for match in matchDate:
    matchDatetime = " ".join(match.text.split())
    matchDate = matchDatetime.split(" ")[0].split("/")[2]
    matchMonth = matchDatetime.split(" ")[0].split("/")[1]
    matchYear = matchDatetime.split(" ")[0].split("/")[0]
    dateMatch.append(matchDate + "/" + matchMonth + "/" + matchYear)
    
    matchHour = int(matchDatetime.split(" ")[1].split(":")[0])
    matchMin = matchDatetime.split(" ")[1].split(":")[1]
    if matchDatetime.split(" ")[2] == "pm" and matchHour != 12:
        matchHour += 12
    
    timeMatch.append(str(matchHour) + ":" + matchMin)

print(dateMatch)
print(timeMatch)

50
['06/04/2025', '30/03/2025', '23/03/2025', '01/02/2025', '26/01/2025', '20/01/2025', '22/12/2024', '21/12/2024', '20/12/2024', '19/12/2024', '06/12/2024', '01/12/2024', '30/11/2024', '29/11/2024', '28/11/2024', '25/11/2024', '24/11/2024', '24/11/2024', '21/11/2024', '29/09/2024', '11/08/2024', '09/08/2024', '04/08/2024', '20/07/2024', '14/07/2024', '13/07/2024', '07/07/2024', '05/07/2024', '01/07/2024', '22/06/2024', '16/06/2024', '03/06/2024', '01/06/2024', '31/05/2024', '12/05/2024', '05/05/2024', '04/05/2024', '03/05/2024', '27/04/2024', '23/04/2024', '21/04/2024', '13/04/2024', '07/04/2024', '23/03/2024', '23/03/2024', '21/03/2024', '18/03/2024', '18/03/2024', '15/03/2024', '25/02/2024']
['15:00', '15:00', '15:00', '19:10', '18:00', '18:15', '17:00', '17:00', '17:00', '17:00', '18:30', '16:00', '16:00', '16:00', '16:00', '18:30', '19:00', '14:30', '18:15', '12:30', '17:15', '17:25', '15:00', '15:00', '18:15', '15:00', '18:00', '15:00', '15:00', '18:50', '18:30', '15:00', '18:00'

In [33]:
matchDiv = soupPrxPlyr.find_all("div", class_="m-item-result")
# matchDiv

iTemps = soupPrxPlyr.find_all("i", class_="fa")
for iTemp in iTemps:
    iTemp.decompose()
    
matchDiv = soupPrxPlyr.find_all("div", class_="m-item-result")
for matchRes in matchDiv:
    matchScores = matchRes.find_all("span")
    matchScore_lst = []
    for score in matchScores:
        matchScore_lst.append(score.text)
    print(matchScore_lst)
# matchDiv

['1', '2']
['1', '2']
['1', '2']
['0', '2']
['2', '0']
['1', '2']
['3', '0']
['2', '0']
['2', '0']
['0', '2']
['1', '2']
['3', '1']
['3', '0']
['0', '2']
['2', '1']
['2', '1']
['2', '1']
['1', '0']
['1', '2']
['1', '0']
['1', '2']
['2', '0']
['0', '2']
['2', '3']
['2', '0']
['0', '2']
['2', '1']
['2', '1']
['2', '0']
['2', '1']
['2', '0']
['1', '2']
['2', '0']
['1', '2']
['3', '2']
['2', '0']
['2', '1']
['2', '1']
['2', '0']
['2', '0']
['0', '2']
['2', '1']
['2', '0']
['1', '3']
['2', '0']
['0', '2']
['2', '1']
['2', '1']
['1', '2']
['1', '3']


In [34]:
teamNames = soupPrxPlyr.find_all("span", class_="m-item-team-name")
for i in range(len(teamNames)):
    if i % 2:
        print("Opp Name: " + " ".join(teamNames[i].text.split()))
    else:
        print("Team Name: " + " ".join(teamNames[i].text.split()))

Team Name: Paper Rex
Opp Name: BOOM Esports
Team Name: Paper Rex
Opp Name: Gen.G
Team Name: Paper Rex
Opp Name: DRX
Team Name: Paper Rex
Opp Name: DetonatioN FocusMe
Team Name: Paper Rex
Opp Name: ZETA DIVISION
Team Name: Paper Rex
Opp Name: T1
Team Name: Paper Rex x Spotlight Series
Opp Name: Xipto x Global Esports
Team Name: Paper Rex x Spotlight Series
Opp Name: ZETA DIVISION GC
Team Name: Paper Rex x Spotlight Series
Opp Name: DRX Changers
Team Name: Paper Rex x Spotlight Series
Opp Name: Xipto x Global Esports
Team Name: Paper Rex
Opp Name: Wolves Esports
Team Name: Paper Rex
Opp Name: DetonatioN FocusMe
Team Name: Paper Rex
Opp Name: Trace Esports
Team Name: Paper Rex
Opp Name: DetonatioN FocusMe
Team Name: Paper Rex
Opp Name: DRX
Team Name: Paper Rex
Opp Name: Rex Regum Qeon
Team Name: Paper Rex
Opp Name: EDward Gaming
Team Name: Team DDPY
Opp Name: Team FWA
Team Name: Paper Rex
Opp Name: Gen.G
Team Name: Team f0rsakeN
Opp Name: Team xccurate
Team Name: Paper Rex
Opp Name: EDwar

In [35]:
resTest = soupPrxPlyr.find_all('div', class_="m-item-result")
for res in resTest:
    print(res.get("class")[1].split("-")[1])

loss
loss
loss
loss
win
loss
win
win
win
loss
loss
win
win
loss
win
win
win
win
loss
win
loss
win
loss
loss
win
loss
win
win
win
win
win
loss
win
loss
win
win
win
win
win
win
loss
win
win
loss
win
loss
win
win
loss
loss


In [36]:
soupMatch = BeautifulSoup(requests.get("https://www.vlr.gg/458813/gen-g-vs-paper-rex-champions-tour-2025-pacific-stage-1-w2").content, 'html.parser')
titleMatch = " ".join(soupMatch.find_all('title')[0].text.split()).split("|")
for sub in titleMatch:
    print(sub.strip())

Gen.G vs. Paper Rex
Champions Tour 2025: Pacific Stage 1
Group Stage
Valorant match
VLR.gg


In [37]:
soupMatch = BeautifulSoup(requests.get("https://www.vlr.gg/458813/gen-g-vs-paper-rex-champions-tour-2025-pacific-stage-1-w2").content, 'html.parser')
matchStats = soupMatch.find_all('td', class_="mod-stat")
for stat in matchStats:
    stat.decompose()
matchTable = soupMatch.find_all('div', class_="vm-stats-game mod-active")
for matchPlayer in matchTable:
    player = matchPlayer.find_all('td', class_="mod-player")
    for playerName in player:
        name = playerName.find('div', class_="text-of")
        print("".join(name.text.split()))

Karon
t3xture
Munchkin
Ash
Suggest
f0rsakeN
something
Jinggg
mindfreak
d4v41


In [38]:
soupMatch = BeautifulSoup(requests.get("https://www.vlr.gg/458813/gen-g-vs-paper-rex-champions-tour-2025-pacific-stage-1-w2").content, 'html.parser')
# soupMatch
matchPlayer = soupMatch.find_all('td', class_="mod-player")
for stat in matchPlayer:
    stat.decompose()
matchTable = soupMatch.find_all('div', class_="vm-stats-game mod-active")
matchTable

[<div class="vm-stats-game mod-active" data-game-id="all">
 <div style="text-align: right; margin-top: 5px; ">
 <div class="wf-filter-inset js-side-filter noselect" style="margin-bottom: 0px;">
 <div class="mod-active" data-side="both">All</div><div data-side="t">Attack</div><div data-side="ct">Defend</div>
 </div>
 </div>
 <div>
 <div style="overflow-x: auto; margin-top: 15px; padding-bottom: 5px;">
 <table class="wf-table-inset mod-overview">
 <thead>
 <tr>
 <th></th>
 <th title="Agent"></th>
 <th title="Rating 2.0"><span>R<sup>2.0</sup><i></i></span></th>
 <th title="Average Combat Score"><span>ACS<i></i></span></th>
 <th style="padding-left: 14px;" title="Kills"><span>K<i></i></span></th>
 <th title="Deaths"><span>D<i></i></span></th>
 <th title="Assists"><span>A<i></i></span></th>
 <th title="Kills - Deaths"><span>+/–<i></i></span></th>
 <th "="" title="Kill, Assist, Trade, Survive %"><span>KAST<i></i></span></th>
 <th title="Average Damage per Round"><span>ADR<i></i></span></th>
